* Merhaba arkadaşlar.Bu notebook içerisinde BPE formatında tokenizer yazmayı öğreneceğiz.(Tokenizer.ipynb) notebook içeriğinden genel anlatımlara erişebilirsiniz.Kısa bir teorikle işe başlayalım.

# SentencePiece-BPE Tabanlı Tokenizer Pipeline  
*Konuşmacı & Stil Etiketleri, UNK-Dropout Augmentasyonu*  



## 1. Teorik Arka Plan — BPE (Byte-Pair Encoding)

**Byte-Pair Encoding**, bir karakter çifti sık sık art arda görüldüğünde bu çifti tek bir “parça” olarak sözlüğe ekleyen, yinelemeli (iterative) bir sıkıştırma-esinli algoritmadır.  
Neden tercih edilir?

| Özellik | Açıklama |
|---------|----------|
| **OOV Sorununu Azaltır** | Yeni kelimeler, mevcut alt-parçalara bölünerek temsil edilir. |
| **Küçük Sözlük** | Binlerce nadir kelime yerine, birkaç bin alt-parça yeterli olur. |
| **Dil Bağımsız** | Eklemeli diller (Türkçe gibi) dahil olmak üzere her dilde çalışır. |
| **Modern Modellerle Uyum** | Transformer, BERT, GPT vb. modeller BPE/WordPiece tabanlıdır. |

Bu projede **SentencePiece** kütüphanesi kullanılarak ~1 800 parçalık bir Türkçe BPE sözlüğü eğitilmiştir.  
Ek olarak:

- `<pad>`, `<sos>`, `<eos>` → sıra kontrol token’ları  
- `<user>`, `<bot>` → konuşmacı rolleri  
- `<casual>` → stil/ton etiketi  
- UNK-Dropout (%10) → rastgele token’ları `<unk>` ile değiştirerek dayanıklılık (robustness) artırır.


## Neden “Klasik Tokenizer”dan Farklı?  
### (Konuşmacı & Stil Etiketleri + SentencePiece-BPE + UNK-Dropout)



### 1. Klasik Tokenizer’ın Sınırı  
`Tokenizer.fit_on_texts()` **kelime-frekans tabanlı** bir yöntemdir:

1. Metni boşluklara göre böler.  
2. En sık kelimeleri sözlüğe alır, geri kalanını `<unk>` yapar.  

Bu yapı;  
- OOV (Out-of-Vocab) sorununu tam çözmez,  
- Konuşmacı rolü / ton kontrolü gibi diyaloğa özgü sinyalleri içermez,  
- Her kelimeyi tek parça tuttuğu için eklemeli dillerde (Türkçe) çok büyük vocab oluşturur.



### 2. Subword-BPE Neden Farklıdır?  

| Özellik | Klasik Kelime Tabanlı | BPE / Subword |
|---------|-----------------------|---------------|
| **Eğitim** | Sadece `fit_on_texts` (sıklık) | Ayrı **model eğitimi** (SentencePiece) |
| **Token Birimi** | Tam kelime | Sık parçalar + kalan karakter dizileri |
| **OOV** | Yüksek | Pratikte sıfıra yakın |
| **Vocab Boyutu** | Kelime sayısına bağlı (çok büyük) | Aynı kapsama için küçük (∼k) |

> **Farklılık:** Subword modeli, klasik tokenizer’dan önce **bir kere** korpus üzerine “öğrenilir” ve `.model` dosyası üretir. Bu ekstra adım klasik yolu ayıran temel noktadır.



### 3. Konuşmacı & Stil Etiketleri  

- **Amaç:** Modelin diyaloğu “kim söylüyor” (user/bot) ve “hangi ton” (casual/formal) ile ürettiğini kavraması.  
- **Yöntem:** Ham cümle başına etiketi metnin **bir parçası** olarak eklemek.  


user> casual> merhaba

bot> <casual  merhaba, nasıl yardımcı olabilirim?

- **Farklılık:** Bu etiketler **dilsel anlam** taşımaz; tokenizer’ın kendiliğinden çıkaramayacağı, tasarımcı tarafından eklenmiş “kontrol tokenları”dır.



### 4. UNK-Dropout (Token-Level Data Augmentation)  

| Aşama | Klasik Tokenizer | Bu Çalışma |
|-------|------------------|------------|
| **Veri Hazırlığı** | Diziye çevir → pad | Diziye çevir → **rastgele %α token’ı `<unk>` yap** → pad |

- **Amaç:** Modeli rastgele OOV’lerle tanıştırarak üretim sırasında beklenmedik kelimelere karşı sağlamlaştırmak.  
- **Farklılık:** Bu adım *tokenizer*’ın hemen arkasında, `tf.data.Dataset.map()` içinde yapılır; klasik yolda hiç yoktur.



### 5. Özet  

| Katman | Klasik Yaklaşım | Gelişmiş (Bu Çalışma) |
|--------|-----------------|-----------------------|
| **Temizleme** | Basit regex | Aynı |
| **Vocab Eğitimi** | `fit_on_texts` | *SentencePiece-BPE* (dış model) |
| **Kontrol Tokenları** | Yok | `<user>`, `<bot>`, `<casual>` |
| **Augmentation** | Yok | UNK-Dropout |
| **Çıktı** | Kelime ID dizisi | Subword ID dizisi + rol/ton sinyali |

Bu üç yenilik — **BPE**, **konuşmacı/ton token’ları**, **UNK-dropout** — klasik tokenizer’ın ötesine geçerek diyaloğa özgü kontrol, düşük OOV ve artırılmış dayanıklılık sağlar.  


### KODA UYGULAMA ADIMINA GEÇELİM..

In [16]:
"""
SENTENCEPIECE-BPE TOKENIZER PIPELINE  (Chatbot sürümü)
───────────────────────────────────────────────────────
• Özel semboller: <pad> <sos> <eos> <user> <bot> <casual>
• Subword sözlük ≈1 800 parça  (küçük korpus)
• UNK-Dropout (%10)  →  veri tabanlı regularization
• tf.data.Dataset  →  shuffle • batch • prefetch
(Masking/model tarafı sonraki notebook’a bırakıldı)
"""

'\nSENTENCEPIECE-BPE TOKENIZER PIPELINE  (Chatbot sürümü)\n───────────────────────────────────────────────────────\n• Özel semboller: <pad> <sos> <eos> <user> <bot> <casual>\n• Subword sözlük ≈1 800 parça  (küçük korpus)\n• UNK-Dropout (%10)  →  veri tabanlı regularization\n• tf.data.Dataset  →  shuffle • batch • prefetch\n(Masking/model tarafı sonraki notebook’a bırakıldı)\n'

In [17]:
# ───────────────────────────────────────────────────────────────
# 0) Gereksinimler
#    pip install sentencepiece pandas tensorflow==2.18
# ───────────────────────────────────────────────────────────────
import io, re, unicodedata, sentencepiece as spm
import pandas as pd, tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [18]:
df = pd.read_csv(r"C:\Users\hdgn5\OneDrive\Masaüstü\Transformerlar\Konu Anlatımları\Encoder - Decoder - PE - ATTN ( İLERİ DÜZEY ) = TF\Tokenizer\örnek_set .csv")

raw_inputs = df["input"].astype(str).to_list()
raw_targets = df["output"].astype(str).to_list()

In [19]:
def clean_texts(text:str) -> str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

In [20]:
USER , BOT , CASUAL = "<user>" , "<bot>" , "<casual>"

input_texts = [f"{USER} {CASUAL} {clean_texts(t)}" for t in raw_inputs]
target_texts = [f"{BOT} {CASUAL} {clean_texts(t)}" for t in raw_targets]

In [21]:
# ───────────────────────────────────────────────────────────────
# 2) SentencePiece-BPE Eğitimi
# ───────────────────────────────────────────────────────────────

MODEL_PREFİX , VOCAB_DESIRED , VOCAB_LIMIT = "bpe_tr" , 8000 , 1000

with io.open("corpus.txt" , "w" ,  encoding="utf-8") as f:
    f.write("\n".join(  input_texts+target_texts))

In [22]:
spm.SentencePieceTrainer.Train(
    input = "corpus.txt",
    model_prefix = MODEL_PREFİX,
    model_type = "bpe",
    vocab_size = min(VOCAB_DESIRED,VOCAB_LIMIT),
    user_defined_symbols =
    [
     "<pad>" , "<sos>" , "<eos>" ,
     USER,BOT,CASUAL
    ]
)

In [23]:
sp = spm.SentencePieceProcessor(model_file=f"{MODEL_PREFİX}.model")
pad_id  = sp.piece_to_id("<pad>")
sos_id  = sp.piece_to_id("<sos>")
eos_id  = sp.piece_to_id("<eos>")
unk_id = sp.unk_id()
vocab_size = sp.get_piece_size()

In [24]:
# ───────────────────────────────────────────────────────────────
# 3) Encode & Pad
# ───────────────────────────────────────────────────────────────

enc   = [sp.encode(t, out_type=int)                for t in input_texts]
din   = [[sos_id] + sp.encode(t, out_type=int)     for t in target_texts]
dout  = [sp.encode(t, out_type=int) + [eos_id]     for t in target_texts]

In [25]:
max_len = max(max(map(len,enc)) , max(map(len,din)) , max(map(len,dout)))
print(max_len)

16


In [26]:
enc_in = pad_sequences(enc,maxlen=max_len ,padding="post" , value=pad_id)
dec_in = pad_sequences(din,maxlen=max_len , padding="post" , value=pad_id)
dec_out = pad_sequences(dout,maxlen=max_len,padding="post" , value=pad_id)

In [4]:
DROP_RATE = 0.10

In [5]:
def unk_dropout(inputs,targets,rate=DROP_RATE,unk=unk_id):
    e = inputs["encoder_input"]
    d = inputs["decoder_input"],

    mask_e = tf.cast(tf.random.uniform(tf.shape(e)) < rate , e.dtype)
    mask_d = tf.cast(tf.random.uniform(tf.shape(d)) < rate , d.dtype)

    inputs["encoder_input"] = tf.where(mask_e == 1 , unk , e )
    inputs["decoder_input"] = tf.where(mask_d == 1 , unk , d)

    return inputs , targets


NameError: name 'unk_id' is not defined

In [6]:
def schedule(ep):
    if ep < 5:   return 0.05
    if ep < 15:  return 0.12
    return 0.08

class UnkScheduler(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        rate = schedule(epoch)
        DROP_RATE.assign(rate)
        print(f"[UNK-Dropout] epoch {epoch} → rate={rate:.2f}")


NameError: name 'tf' is not defined

In [7]:
# ───────────────────────────────────────────────────────────────
# 5) tf.data.Dataset Pipeline
# ───────────────────────────────────────────────────────────────
BATCH_SIZE  = 32    # her adımda GPU’ya gidecek örnek sayısı
BUFFER_SIZE = 1000  # shuffle havuzu (rastgelelik için)

dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            {
                "encoder_input": enc_in,   # ← modelin encoder girdisi
                "decoder_input": dec_in    # ← decoder girdisi (<sos> + hedef)
            },
            dec_out                       # ← decoder hedef dizisi (<eos> ekli)
        )
    )
    # 1️⃣  Rastgele %10 token’ı <unk> yap → dayanıklılık artır
    .map(unk_dropout, num_parallel_calls=tf.data.AUTOTUNE)

    # 2️⃣  BUFFER_SIZE kadar örnek bellekte karıştır → her epoch farklı sıra
    .shuffle(BUFFER_SIZE)

    # 3️⃣  Sabit boyutlu kümeler oluştur; eksik son batch’i at
    .batch(BATCH_SIZE, drop_remainder=True)

    # 4️⃣  Eğitim adımı GPU’da çalışırken bir sonraki batch’i hazırla
    .prefetch(tf.data.AUTOTUNE)
)

NameError: name 'tf' is not defined

In [30]:
# --------------------- Yüzdesel Bilgi Log’u ----------------------------------
total_tokens   = (enc_in != pad_id).sum() + (dec_in != pad_id).sum()
unk_tokens_est = int(total_tokens * DROP_RATE)    # teorik beklenti
print(f"UNK-Dropout Rate: {DROP_RATE*100:.0f}% → ~{unk_tokens_est:,} token her epoch’ta <unk> olacak")

print("Dataset hazır →", next(iter(dataset.take(1)))[0]["encoder_input"].shape)
print(f"pad:{pad_id}  sos:{sos_id}  eos:{eos_id}  unk:{unk_id}  vocab:{vocab_size}")
print("max_len:", max_len)

UNK-Dropout Rate: 10% → ~92 token her epoch’ta <unk> olacak
Dataset hazır → (32, 16)
pad:3  sos:4  eos:5  unk:0  vocab:1000
max_len: 16


## UNK-Dropout Oranı (0.10) — Yüksek mi, Düşük mü?

### 1. Ne Yapıyoruz?
`DROP_RATE = 0.10` ⇒ **her epoch** sırasında, pad-dışı token’ların yaklaşık %10’u rastgele `<unk>` (OOV) olarak maskeleniyor.  
Bu, modelin “eksik veya bilinmeyen” kelimelerle karşılaştığında paniğe kapılmamasını sağlar (regularization).


### 2. Oran Çok **Düşük** Olursa (≤ 0.02)
| Sonuç | Açıklama |
|-------|----------|
| Zayıf Etki | Model neredeyse her zaman gerçek kelimeleri görür → OOV senaryolarına hâlâ kırılgan kalabilir. |
| Overfitting Riski | Düzenleyici (noise) çok az → aşırı ezberleme sürer. |



### 3. Oran Çok **Yüksek** Olursa (≥ 0.20)
| Sonuç | Açıklama |
|-------|----------|
| Bilgi Kaybı | Cümlenin anlamı bozulur; model yeterli bağlam göremez. |
| Eğitim Yavaşlar | Daha fazla adım gerekir, valid/perf metriği düşebilir. |
| “Dil” Yıpranması | Özellikle kısa cümlelerde her kelimenin `<unk>` olma riski artar. |



### 4. Pratik Öneri  
| Veri Boyutu | Önerilen Aralık |
|-------------|-----------------|
| 10 k – 100 k cümle | **0.05 – 0.10** |
| 100 k – 1 M cümle  | **0.10 – 0.15** |
| +1 M cümle         | 0.15’e kadar çıkılabilir; 0.20 genelde üst sınır |

> **Başlangıç için** `0.10` (yani %10) yaygın ve güvenli bir değerdir.  
> Eğitim/valid loss eğrilerini izleyin:  
> • Çok oynaklık varsa oranı **azaltın**.  
> • Valid-loss ↗️ ama train-loss ↘️ ise overfitting var demektir → oranı **artırın** (0.12–0.15).  


### 5. Kademeli (Schedule) Stratejisi  
- **İlk epoch’lar**: düşük (0.05) — model temeli öğrensin.  
- **Sonraki epoch’lar**: orta (0.10) — dayanıklılığı artır.  
- **Fine-tuning** aşamasında: tekrar düşür (0.02–0.05) — son hassasiyet.


**Özet:** `0.10` tipik ve dengeli bir seçenektir. Çok düşük oran düzenleyici etkisini zayıflatır; çok yüksek oran ise anlamlı bağlamı aşındırır. Oranı veri boyutu, cümle uzunluğu ve valid-loss davranışına göre ayarlayın.


-------
-------

##### Aslında yapılan bu işlemler kelime bazlı tokenizer işleminde fazlasıyla benzer.Orda yapılan elle token ekleme işlemlerini burda da yapacağız.Unk dropout gibi tabirleri burda da yeteri kadar önemle kullanacağız.Kelime bazlı tokenizer ı eğer anlayabildiyseniz diğer işlemleri de aynı şekilde anlayacaksınız.Şimdi yavaştan bu işlemlere geçelim.

##  1.CSV Okuma

In [31]:
df = pd.read_csv(r"C:\Users\hdgn5\OneDrive\Masaüstü\Transformerlar\Konu Anlatımları\Encoder - Decoder - PE - ATTN ( İLERİ DÜZEY ) = TF\Tokenizer\örnek_set .csv")
df.head()

,input,output
0,Merhaba,"Merhaba, size nasıl yardımcı olabilirim?"
1,Nasılsın?,"İyiyim, teşekkür ederim. Siz nasılsınız?"
2,Adın ne?,Ben bir yapay zekâ asistanıyım. Adım yok ama y...
3,Kaç yaşındasın?,"Benim yaşım yok, dijitalim!"
4,Bugün günlerden ne?,"Maalesef tarih bilgim yok, ama sistem saatinde..."


In [32]:
raw_inputs = df["input"].astype(str).to_list()
raw_targets = df["output"].astype(str).tolist()

## 2. Metin Temizleme

In [33]:
def clean_texts(text:str) -> str:
    text = unicodedata.normalize("NFC",text).lower()
    text = re.sub(r"[^a-zçğıöşüğ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

## User Bot ve Casual Etiketleri


In [34]:
user,bot,style = "<user>" , "<bot>" , "<casual>" 

input_texts = [f"{user} {style} {clean_texts(t)}" for t in raw_inputs]
targets_text = [f"{bot} {style} {clean_texts(t)}" for t in raw_targets] 

## Corpus.txt dosyasını hazırlamak

### Neden “corpus.txt” Dosyası Oluşturuyoruz?

* SentencePiece’in BPE modelini eğitmek için **saf metin dosyası** (`.txt`) formatında bir girdi ister. Python’daki listeler veya DataFrame doğrudan kullanılamaz:

* input_texts ve target_texts listelerindeki her cümleyi tek tek satır olarak dosyaya yazar.



* Özet: Bu adım, BPE sözlük üretimi için zorunlu bir ön hazırlıktır—SentencePiece eğitim rutini metni dosyadan okur, belleğe değil.

In [35]:
MODEL_PREFİX , VOCAB_DESIRED , VOCAB_LIMIT = "bpe_tr" , 8000 , 1000

with io.open("corpus.txt" , "w" ,  encoding="utf-8") as f:
    f.write("\n".join(  input_texts+targets_text))

In [36]:
import sentencepiece as sp

## BPE Modelini Eğitmek İçin `SentencePieceTrainer.Train` Çağrısı

### Alt-Parça Sözlüğü (Subword Vocabulary) Oluşturma

* BPE, sık görülen karakter dizilerini “parça” (piece) olarak modelleyerek OOV sorununu büyük ölçüde ortadan kaldırır.

* Bu fonksiyon, kelime düzeyinde değil, subword düzeyinde bir sözlük üretir.

### Parametrelerin Anlamı

* input="corpus.txt"

Eğitim verisi olarak kullanılacak ham metin dosyasının yolu.

* model_prefix=MODEL_PREFIX

Oluşacak model dosyasının (“bpe_tr.model” / “bpe_tr.vocab”) önek adı.

* model_type="bpe"

Byte-Pair Encoding algoritmasını seçer.

* vocab_size=…

Kaç subword parçası (ör. 1 800) eğitileceğini sınırlar.

* user_defined_symbols=[…]

pad>, <sos, eos> gibi özel token’ları ve konuşmacı/stil etiketlerini (USER, BOT, STYLE) sözlükte sabit tutar.

In [37]:

sp.SentencePieceTrainer.Train(
    input="corpus.txt",
    model_prefix = MODEL_PREFİX,
    model_type = "bpe",
    vocab_size = min(VOCAB_DESIRED,VOCAB_LIMIT),
    user_defined_symbols=[
        "<pad>" , "<sos>" , "<eos>", user , bot , style
        ]
    )

## `SentencePieceProcessor` ile Özel Token ID’lerini Alma

Bu kod bloğunda etiketleme değil, yüklenen BPE modelindeki özel token’ların ID’lerini alıyoruz:

* spm.SentencePieceProcessor(...)

Önceki adımda eğittiğimiz .model dosyasını yükler.

* piece_to_id("pad>"), …

pad>, <sos, eos> gibi kontrol token’larının sayısal ID değerlerini döner.

* unk_id = sp.unk_id()

Modelin varsayılan OOV (unk) ID’sini alır.

* vocab_size = sp.get_piece_size()

Toplam parça sayısını (vocab büyüklüğü) öğrenir.

In [38]:
sp = spm.SentencePieceProcessor(model_file = f"{MODEL_PREFİX}.model")

pad_id = sp.piece_to_id("<pad>")
sos_id = sp.piece_to_id("<sos>")
eos_id = sp.piece_to_id("<eos>")
unk_id = sp.unk_id()
vocab_size = sp.get_piece_size()

## Encoder ve Pad

In [40]:
enc = [sp.encode(t,out_type=int) for t in input_texts]
din = [[sos_id] + sp.encode(t,out_type=int) for t in target_texts]
dout = [sp.encode(t,out_type=int) + [eos_id] for t in target_texts]

## Max_len Belirleme

In [41]:
max_len = max(max(map(len,enc)) , max(map(len,din)) , max(map(len,dout)))
print(max_len)

16


## Pad_Sequences

In [43]:
enc_in = pad_sequences(enc,maxlen=max_len,padding="post",value=pad_id)
dec_in = pad_sequences(din,maxlen=max_len,padding="post" , value=pad_id)
dec_out = pad_sequences(dout,maxlen=max_len,padding="post",value=pad_id)

# value=pad_id derken kullandığımız sayı pad_id, BPE sözlüğünde <pad> token’ına karşılık gelen sayısal ID’dir.

## UNK DROPOUT

In [44]:
DROP_RATE =0.10

def unk_dropout(inputs,targets,rate=DROP_RATE,unk=unk_id):
    e = inputs["encoder_input"]
    d = inputs["decoder_input"]

    mask_e = tf.cast(tf.random.uniform(tf.shape(e)) < rate , e.dtype)
    mask_d = tf.cast(tf.random.uniform(tf.shape(d)) < rate ,d.dtype)

    e = tf.where(mask_e == 1 , unk ,e)
    d = tf.where(mask_d == 1 ,unk , d)

    return inputs , targets

In [45]:
def schedule(epochs):

    if epochs < 5 : return 0.05
    if epochs < 15 : return 0.12 

    return 0.08 

class UnkScheduler(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs=None):
        rate = schedule(epoch)
        DROP_RATE.assign(rate)
        print(f"[UNK_DROPOUT] epoch {epoch} -> rate={rate:.2f}")

## Dataset Pipeline

In [46]:
buffer = 1000
batch = 32

dataset =(
    tf.data.Dataset.from_tensor_slices(
        (
        {
            "encoder_input": enc_in,
            "decoder_input" : dec_in
            },dec_out
        )
    )
    .map(unk_dropout,num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch,drop_remainder=True)
    .shuffle(buffer)
    .prefetch(tf.data.AUTOTUNE)
)

## Yüzdesel Bilgi Logu

In [47]:
total_tokens = (enc_in !=pad_id).sum()+(dec_in != pad_id).sum()
unk_tokens_est = int(total_tokens * DROP_RATE)

In [48]:
print(f"UNK-Dropout Rate: {DROP_RATE*100:.0f}% → ~{unk_tokens_est:,} token her epoch’ta <unk> olacak")

print("Dataset hazır →", next(iter(dataset.take(1)))[0]["encoder_input"].shape)
print(f"pad:{pad_id}  sos:{sos_id}  eos:{eos_id}  unk:{unk_id}  vocab:{vocab_size}")
print("max_len:", max_len) 

UNK-Dropout Rate: 10% → ~92 token her epoch’ta <unk> olacak
Dataset hazır → (32, 16)
pad:3  sos:4  eos:5  unk:0  vocab:1000
max_len: 16
